In [ ]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train['Sex_binario'] = train['Sex'].map({'male':0, 'female':1})

In [4]:
test['Sex_binario'] = test['Sex'].map({'male':0, 'female':1})

In [5]:
# selecione as colunas que são numericos, menos passengerid, survived
X = train.select_dtypes(include=np.number).drop(['PassengerId', 'Survived'], axis=1).fillna(0)
y = train['Survived']

In [8]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.pipeline import make_pipeline

In [12]:
kf = KFold(n_splits=2, random_state=0,shuffle=True)

In [13]:
# criação de matriz de zeros, com o formatado de numeros de linhas com 4 colunas (numero do modelo)
second_level = np.zeros((X.shape[0], 4))

In [16]:
for tr, ts in kf.split(X, y):
    Xtr, Xval = X.iloc[tr], X.iloc[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    # n_jobs = 6 nucleos
    rf  = RandomForestClassifier(n_estimators=100, n_jobs=6, random_state=10)
    rf.fit(Xtr, ytr)
    # probabilidade de um eveneto pertencer a classe 1 ou a classe 0
    prf = rf.predict_proba(Xval)[:,1]
    prf_ = (prf > 0.5).astype(int)
    
    print(f'RF Acurácia: {accuracy_score(yval, prf_)} - Log Loss: {log_loss(yval, prf)}')    
    
    et = ExtraTreesClassifier(n_estimators=100, n_jobs=6, random_state=10)
    et.fit(Xtr, ytr)
    pet = et.predict_proba(Xval)[:,1]
    pet_ = (pet > 0.5).astype(int)
    
    print(f'ET Acurácia: {accuracy_score(yval, pet_)} - Log Loss: {log_loss(yval, pet)}')
    
    lr1 = make_pipeline(StandardScaler(), LogisticRegression())
    lr1.fit(Xtr, ytr)
    plr1 = lr1.predict_proba(Xval)[:,1]
    plr1_1 = (plr1 > 0.5).astype(int)
    
    print(f'LR StdScaler Acurácia: {accuracy_score(yval, plr1_1)} - Log Loss: {log_loss(yval, plr1)}')
        
    lr2 = make_pipeline(MinMaxScaler(), LogisticRegression())
    lr2.fit(Xtr, ytr)
    plr2 = lr2.predict_proba(Xval)[:,1]
    plr2_ = (plr2 > 0.5).astype(int)
    
    print(f'LR MinMax Acurácia: {accuracy_score(yval, plr2_)} - Log Loss: {log_loss(yval, plr2)} \n')
    
    # matriz com as previsões
    
    
    second_level[ts, 0] = prf
    second_level[ts, 1] = pet
    second_level[ts, 2] = plr1
    second_level[ts, 3] = plr2
    

RF Acurácia: 0.7869955156950673 - Log Loss: 0.8891877224352214
ET Acurácia: 0.7802690582959642 - Log Loss: 1.7277767849421113
LR StdScaler Acurácia: 0.7713004484304933 - Log Loss: 0.4548870233895985
LR MinMax Acurácia: 0.773542600896861 - Log Loss: 0.45730271249069515 

     Pclass   Age  SibSp  Parch     Fare  Sex_binario
3         1  35.0      1      0  53.1000            1
4         3  35.0      0      0   8.0500            0
7         3   2.0      3      1  21.0750            0
9         2  14.0      1      0  30.0708            1
11        1  58.0      0      0  26.5500            1
..      ...   ...    ...    ...      ...          ...
873       3  47.0      0      0   9.0000            0
874       2  28.0      1      0  24.0000            1
879       1  56.0      0      1  83.1583            1
884       3  25.0      0      0   7.0500            0
888       3   0.0      1      2  23.4500            1

[445 rows x 6 columns]
     Pclass   Age  SibSp  Parch     Fare  Sex_binario
0  

In [ ]:
for tr, ts in kf.split(X,y):
    Xtr, Xval = second_level[tr], second_level[ts]
    ytr, yval = y.iloc[tr], y.iloc[ts]
    
    lr_stack = LogisticRegression(C=1.)
    lr_stack.fit(Xtr, ytr)
    plr_stack = lr_stack.predict_proba(Xval)[:,1]
    plr_stack_ = (plr_stack > 0.5).astype(int)
    
    print(f'Stack Acurácia: {accuracy_score(yval, plr_stack_)} - Log Loss: {log_loss(yval, plr_stack)} \n')

In [ ]:
pd.DataFrame(np.corrcoef(second_level.T))

In [ ]:

second_level